In [43]:
import pickle
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm  # Import tqdm for progress tracking
from numba import njit, prange
from scipy.spatial.distance import cdist
# numpy.set_printoptions(threshold=sys.maxsize)

In [2]:
worm = 'sub-20190928-03' # Neel
# worm = 'sub-20190928-11' # Maren
# worm = 'sub-20190929-06' # Nalini
# worm = 'sub-20191030-07' # Ryan

with open(f"/scratch/mie8014/BAN/computer_vision/BrainAlignNet/EY/fold1_lrlarge_2/registered_outputs_{worm}.pkl", 'rb') as f:
    warped_data = pickle.load(f)

mappings = list(warped_data.keys())

# with open('/scratch/nar8991/computer_vision/BrainAlignNet/demo_notebook/registered_outputs.pkl', 'rb') as f:
#     warped_data = pickle.load(f)

filepath = '/scratch/mie8014/BAN/computer_vision/data/EY/fold1/test/'

fixed_images = h5py.File(f"{filepath}/{worm}/fixed_images.h5",'r')
fixed_labels = h5py.File(f"{filepath}/{worm}/fixed_labels.h5",'r')
fixed_rois = h5py.File(f"{filepath}/{worm}/fixed_rois.h5",'r')

# Structure of Warpped data
Worm:

    TimeStampMapping:
    
        warped_moving_image:
        
        warped_moving_roi:

        warped_moving_centroids:

In [44]:
# Heuristic to get the largest ROIs
def get_largest_200(array3d_ROIs):
    unique_ROIs = np.unique(array3d_ROIs)
    ROI_sizes = {}
    for label in unique_ROIs:
        ROI_sizes[label] = len(np.where(array3d_ROIs == label)[0])

    largest_ROIs = dict(sorted(ROI_sizes.items(), key = lambda x: x[1])).keys()
    return list(largest_ROIs)[:200]

# Heuristic to get the most distributed ROIs
def get_widely_distributed_rois(array3d_ROIs, num_rois=200):
    # Get unique ROI labels (excluding background, assuming 0 is background)
    unique_ROIs = np.unique(array3d_ROIs)
    unique_ROIs = unique_ROIs[unique_ROIs != 0]  # Exclude background if needed

    # Compute centroids for each ROI
    centroids = []
    for roi in unique_ROIs:
        indices = np.argwhere(array3d_ROIs == roi)
        centroid = np.mean(indices, axis=0)
        centroids.append(centroid)
    centroids = np.array(centroids)

    # Compute pairwise distances between centroids
    distances = cdist(centroids, centroids)

    # Select ROIs with the greatest distances
    selected_rois = []
    remaining_indices = list(range(len(unique_ROIs)))

    # Start with the first ROI (arbitrary choice)
    selected_indices = [remaining_indices.pop(0)]
    selected_rois.append(unique_ROIs[selected_indices[-1]])

    while len(selected_rois) < num_rois and remaining_indices:
        # Calculate the minimum distance from remaining to selected ROIs
        min_distances = np.min(distances[remaining_indices][:, selected_indices], axis=1)
        
        # Find the ROI with the maximum of these minimum distances
        farthest_index = remaining_indices[np.argmax(min_distances)]
        
        # Add to the selected list
        selected_indices.append(farthest_index)
        selected_rois.append(unique_ROIs[farthest_index])
        
        # Remove from remaining
        remaining_indices.remove(farthest_index)

    return selected_rois

array1_3d = warped_data[mappings[2]]['warped_moving_roi']
array1_3d = array1_3d[array1_3d>0]
# slow_largest = get_largest_200(array1_3d)
distributed_rois = get_widely_distributed_rois(array1_3d)

print(distributed_rois)

[1.0, 425.0, 229.0, 152.0, 128.0, 267.0, 274.0, 237.0, 204.0, 8.0, 223.0, 212.0, 9.0, 233.0, 320.0, 163.0, 392.0, 155.0, 323.0, 293.0, 173.0, 104.0, 55.0, 244.0, 278.0, 321.0, 235.0, 306.0, 57.0, 105.0, 143.0, 232.0, 80.0, 386.0, 181.0, 464.0, 451.0, 178.0, 31.0, 92.0, 336.0, 192.0, 46.0, 58.0, 257.0, 364.0, 70.0, 449.0, 44.0, 361.0, 123.0, 411.0, 261.0, 50.0, 140.0, 456.0, 344.0, 81.0, 156.0, 45.0, 333.0, 60.0, 310.0, 291.0, 401.0, 389.0, 30.0, 134.0, 300.0, 414.0, 226.0, 23.0, 262.0, 355.0, 2.0, 82.0, 35.0, 390.0, 356.0, 249.0, 391.0, 138.0, 431.0, 216.0, 365.0, 277.0, 243.0, 6.0, 314.0, 33.0, 19.0, 221.0, 14.0, 26.0, 450.0, 21.0, 17.0, 239.0, 110.0, 430.0, 346.0, 359.0, 322.0, 266.0, 338.0, 53.0, 423.0, 253.0, 256.0, 442.0, 175.0, 295.0, 131.0, 318.0, 189.0, 119.0, 78.0, 458.0, 302.0, 360.0, 461.0, 332.0, 254.0, 394.0, 460.0, 247.0, 12.0, 290.0, 316.0, 385.0, 211.0, 376.0, 63.0, 207.0, 146.0, 428.0, 331.0, 61.0, 198.0, 366.0, 47.0, 203.0, 193.0, 387.0, 74.0, 172.0, 200.0, 416.0, 440

In [32]:
def get_largest_200(array3d_ROIs):
    unique_ROIs = np.unique(array3d_ROIs)
    ROI_sizes = {}
    for label in unique_ROIs:
        ROI_sizes[label] = len(np.where(array3d_ROIs == label)[0])

    largest_ROIs = dict(sorted(ROI_sizes.items(), key = lambda x: x[1])).keys()
    return list(largest_ROIs)[:200]

@njit(parallel=True)
def IoU_3D(seg1, seg2):
    ROIs1=np.unique(seg1)
    ROIs2=np.unique(seg2)

    # assert(len(ROIs1)==len(ROIs2))

    submatrix = np.zeros((400,400))

    for i, label1 in enumerate(ROIs1):
        if label1>0 and i < 400:
            for j,label2 in enumerate(ROIs2):
                if label2 > 0 and j < 400:
                    intersection = np.where(seg1==label1, np.where(seg2==label2,1,0),0)
                    union=np.where((seg1==label1) | (seg2==label2),1,0)#np.where(seg1==label1, np.where(seg2==label2,1,0),0)
                    submatrix[i,j]=np.sum(intersection)/np.sum(union)

    return submatrix[1:,1:] #exclude background

@njit(parallel=True)
def IoU_3D_numba(seg1, seg2):
    ROIs1 = np.unique(seg1)
    ROIs2 = np.unique(seg2)

    # Filter only positive ROI labels
    ROIs1 = ROIs1[ROIs1 > 0]
    ROIs2 = ROIs2[ROIs2 > 0]

    # Limit to 200 largest ROIs
    ROIs1 = get_largest_200(ROIs1)
    ROIs2 = get_largest_200(ROIs2)

    # Preallocate matrix
    submatrix = np.zeros((len(ROIs1), len(ROIs2)))

    for i in prange(len(ROIs1)):  # Use prange for parallel loops
        label1 = ROIs1[i]
        mask1 = seg1 == label1  # Boolean mask for label1
        for j in range(len(ROIs2)):
            label2 = ROIs2[j]
            mask2 = seg2 == label2  # Boolean mask for label2
            intersection = np.sum(mask1 & mask2)
            union = np.sum(mask1 | mask2)
            if union > 0:
                submatrix[i, j] = intersection / union

    return submatrix

In [22]:
# Define the file path to save the matrix_data in HDF5 format
matrix_data_file = f'{worm}_fast_similarity_matrix.h5'
print(matrix_data_file)
# Check if the file already exists
if os.path.exists(matrix_data_file):
    with h5py.File(matrix_data_file, 'r') as f:
        # Load existing matrix_data from the HDF5 file
        matrix_data = {key: f[key][...] for key in f.keys()}
else:
    matrix_data = {}

print("Total mappings to find matrices for: ", len(mappings))

sub-20190928-03_fast_similarity_matrix.h5
Total mappings to find matrices for:  962


In [6]:
for time_map in tqdm(mappings, desc="Processing Mappings", unit="mapping"):
    if time_map in matrix_data:
        print(f"Skipping {time_map}, already computed.")
        continue
    
    print(f"Started {time_map}")
    
    # Assuming the data is correctly indexed here
    array1_3d = warped_data[time_map]['warped_moving_roi']
    array2_3d = fixed_rois[time_map][:,:,:]
    
    # Compute the IoU matrix (or any relevant function)
    roi_overlap_matrix = IoU_3D_numba(array1_3d, array2_3d)
    
    # Save the computed matrix in the dictionary
    matrix_data[time_map] = roi_overlap_matrix
    
    print(f"Completed {time_map}")
    print("\n\n")
    
    # Save the matrix_data to the HDF5 file after processing this time_map
    with h5py.File(matrix_data_file, 'a') as f:  # Use 'a' to append
        if time_map not in f:  # Check if the dataset exists
            f.create_dataset(time_map, data=roi_overlap_matrix)  # Create dataset if it doesn't exist
        else:
            print(f"Dataset {time_map} already exists, skipping creation.")

print(f"Matrix data saved to {matrix_data_file}")

Processing Mappings:   0%|          | 0/962 [00:00<?, ?mapping/s]

Skipping 0to151, already computed.
Skipping 100to454, already computed.
Skipping 101to773, already computed.
Skipping 102to833, already computed.
Skipping 103to171, already computed.
Skipping 104to671, already computed.
Skipping 105to136, already computed.
Skipping 106to850, already computed.
Skipping 107to602, already computed.
Skipping 108to794, already computed.
Skipping 109to179, already computed.
Skipping 10to586, already computed.
Skipping 110to716, already computed.
Skipping 111to766, already computed.
Skipping 112to495, already computed.
Skipping 113to891, already computed.
Skipping 114to680, already computed.
Skipping 115to560, already computed.
Skipping 116to72, already computed.
Skipping 117to795, already computed.
Skipping 118to43, already computed.
Skipping 119to692, already computed.
Skipping 11to707, already computed.
Skipping 120to46, already computed.
Skipping 121to498, already computed.
Skipping 122to550, already computed.
Skipping 123to835, already computed.
Skipping

Processing Mappings:  21%|██▏       | 206/962 [00:17<01:03, 11.82mapping/s]

Completed 285to691



Started 286to556
Completed 286to556



Started 287to82


Processing Mappings:  22%|██▏       | 208/962 [00:35<02:35,  4.85mapping/s]

...

Completed 98to443



Started 99to412


Processing Mappings: 100%|█████████▉| 961/962 [1:50:48<00:08,  8.81s/mapping]

Completed 99to412



Started 9to414


Processing Mappings: 100%|██████████| 962/962 [1:50:57<00:00,  6.92s/mapping]

Completed 9to414



Matrix data saved to sub-20190928-03_fast_similarity_matrix.h5


# Scrap Work

In [11]:
array1_3d = warped_data[mappings[0]]['warped_moving_roi']
array1_3d.shape

(256, 128, 21)

In [12]:
array2_3d = fixed_rois[mappings[0]][:,:,:]
array2_3d.shape

(256, 128, 21)

In [10]:
ROIs1=np.unique(array1_3d)
ROIs2=np.unique(array2_3d)

In [11]:
%%time
roi_overlap_matrix = IoU_3D(array1_3d, array2_3d)

CPU times: user 3h 4min 43s, sys: 812 ms, total: 3h 4min 44s
Wall time: 16min 57s


In [13]:
%%time
roi_overlap_matrix_with_nmb = IoU_3D_numba(array1_3d, array2_3d)

CPU times: user 1min 45s, sys: 6.98 ms, total: 1min 45s
Wall time: 10.9 s


In [14]:
roi_overlap_matrix_with_nmb

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01712329, 0.00653595,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [18]:
roi_overlap_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01712329,
        0.00653595]])

In [25]:
compare_matrix = roi_overlap_matrix == roi_overlap_matrix_with_nmb[:-1,:-1]
np.all(compare_matrix)

True

In [ ]:
# Reread one last time and remove duplicates

In [ ]:
len(matrix_data)